#CatDog classification Task

이번 실습에서는 Kaggle에서 제공하는 cat & dog classification을 수행할 것입니다. 
<br>
여러분은 이 Task를 수행하기 위한 모든 것을 이미 다 배웠습니다! 무엇이 필요한지는 제가 하나하나 다 말씀 드릴테니 직접 찾아서 수행해보시기 바랍니다

## Dataset Download

해당 데이터셋은 https://www.kaggle.com/competitions/dogs-vs-cats/data 여기에서 찾아보실 수 있습니다. Kaggle API를 이용해서 직접 다운 받아주세요!
<br>
**Hint : basic_wk6**

## Preprocessing

먼저 다운받은 데이터셋을 압축 해제하자.
<br>
**Hint : basic_wk6**

In [ ]:
import zipfile
import os

zip_directory = "C:/Users/Unity/Downloads/dogs-vs-cats"

dataset_directory = "C:/Users/Unity/Downloads/CatDog"


dataset_zip = zipfile.ZipFile(zip_directory)
dataset_zip.extractall(dataset_directory)
dataset_zip.close()

train_zip = zipfile.ZipFile(os.path.join(dataset_directory, 'train.zip'))
train_zip.extractall(dataset_directory)
train_zip.close()

test_zip = zipfile.ZipFile(os.path.join(dataset_directory, 'test1.zip'))
test_zip.extractall(dataset_directory)
test_zip.close()



Custom Dataset 만들기

In [ ]:
import torch.utils.data as data
from torchvision.io import read_image
import torchvision

dataset_len = len(os.listdir(os.path.join(dataset_directory, 'train'))) #25000
test_len = len(os.listdir(os.path.join(dataset_directory, 'test1'))) #12500

dog_dir = [f'dog.{i}.jpg' for i in range(12500)]
cat_dir = [f'cat.{i}.jpg' for i in range(12500)]  #파일 디렉토리가 dog.i.jpg , cat.i.jpg 로 되어있다.

train_annotation = dog_dir + cat_dir                      #같은 폴더에 cat, dog가 동시에 존재
test_annotation = [f'{i+1}.jpg' for i in range(12500)]     # 1.jpg, 2.jpg 로 되어있음

train_dir = os.path.join(dataset_directory, 'train')
test_dir = os.path.join(dataset_directory, 'test1')





class CustomDataset(data.Dataset):
  def __init__(self, files, root, mode='train', transform = None):
    self.files = files
    self.root = root
    self.mode = mode
    self.transform = transform

  def __len__(self):
    return len(self.files)

  def __getitem__(self, index):
    img_path = os.path.join(self.root, self.files[index]) # 'C:/Users/Unity/Downloads/CatDog/train/dog.index.jpg
    image = read_image(img_path)
    if 'cat' in self.files[index]:
      label = 0
    else:
      label = 1

    if self.transform:
      image = self.transform(image)
    
    if self.mode == 'train':
      return image, label
    elif self.mode == 'test':
      return index



디렉토리에 저장되어있는 데이터셋을 데이터로더를 통해 학습시키기 위해 커스텀 데이터셋을 직접 만들어야한다. 코드를 보면 알겠지만
<br>
__len__ : 데이터셋 크기 리턴
<br>
__getitem__ : 메서드가 불릴 때마다 디렉토리에서 이미지 한장과 그 정답을 리턴

이제 transform을 정의하고 DataLoader를 만들자

In [ ]:
import torch

train_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((256,256)),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
])
test_transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((256,256)),
    torchvision.transforms.ToTensor()
])

train_dataset = CustomDataset(train_annotation, train_dir, mode='train', transform = train_transform)
test_datset = CustomDataset(test_annotation, test_dir, mode='test', transform = test_transform)


#==============================================================================
#여기서부터는 Custom Dataset을 토대로 DataLoader를 정의해야합니다.
#이건 이미 다 배운 내용이므로 생략하겠습니다.

#정의되어야할 내용: 무작위 셔플을 설정해주시고, 배치 사이즈는 64로 정해주시고, 
#마지막 64로 나누어떨어지지 않는 배치는 사용하지 않도록 해주세요!
#testloader는 셔플을 하시지 않는 걸 추천드립니당

#Hint : wk8 분류기 훈련시키기
#==============================================================================

pass


## 커스텀 모듈 만들기 feat. Subclass Modeling

CNN 모듈을 이용해서 여러분이 직접 원하는 형태의 인공지능 모델을 생성해주세요!
<br>
Subclass 모델링을 사용해주시기 바랍니다!
<br>
<br>
**HINT: wk8_분류기_훈련시키기 or basic_wk9**
<br>
<br>
더 높은 난이도를 원하신다면 : 모듈을 여러개 만들어서 메인 모듈에서 합쳐보세요!

In [ ]:
import torch.nn as nn

class Model(nn.Module):
  def __init__(self):
    super(Model, self).__init__()

    pass

  def forward(self, x):
    pass
    #모델의 최종 아웃풋은 2차원 벡터입니다

원하는대로 잘 만들었는지 확인해보자.

In [ ]:
from torchsummary import summary

model = Model()
model.cuda()

summary(model, input_size=(3,256,256))

## Training Model

이제 만든 모델을 학습해보자!

**HINT : wk8_분류기_훈련시키기**

In [ ]:
def train(model, epoch, device, dataloader, criterion, optimizer):
    model.train()
    
    for i in range(epoch):
        train_loss = 0
        correct = 0
        total = 0

        for idx, (input, target) in enumerate(dataloader):
            
            
            input = input.to(device)
            target = target.to(device)
            output = model(input)
            loss = criterion(output, target)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

        acc = 100 * correct / total
        print('train epoch : {} [{}/{}]| loss: {:.3f} | acc: {:.3f}'.format(
        i, idx, len(dataloader), train_loss/(idx+1), acc))
    
    print("end of training")


만족할만 했다면 뒤의 test로 넘어가고, 아니라면 training의 하이퍼파라미터나 모델을 수정해보자

## Test

이제 대회에 제출하기 위한 test set을 eval 해보자
<br>
<br>
이미 test에 대한 dataloader가 정의되어 있어야합니다! 아니라면 위의 Dataloader 칸으로 올라가서 testloader를 생성해주세요

In [ ]:
import pandas as pd

def predict(model, testloader, device, submission_dir):
  model.eval()
  sub = pd.read_csv(submission_dir)
  with torch.no_grad():
    for x, index in testloader:
      x.to(device)
      pred = model(x)

      sub.iloc[index,1]=pred.argmax(pred, dim=1).numpy()        #csv 파일에 각각 정답을 입력하는 것.
      
  return sub


아 근데...다하고 보니까 이게 submission이 닫혀있네요...제가 인공지능 처음 배웠을 때는 열려있었는데...ㅎ...죄송함다...submission은 못하지만..그래도 방식은 알려드리겠습니다..!


<img src="https://ifh.cc/g/C3j8Xh.jpg" alt="Alternative text" style="width:400px;" />

이처럼 kaggle 대회 페이지에 가보면 Submit Predictions라는 버튼이 있습니다. 해당 버튼을 누르면 

<img src="https://ifh.cc/g/G4NwBg.png
" alt="Alternative text" style="width:300px;" />

여기에 저장한 submission.csv를 업로드하면 됩니다 아니면 밑의 명령어를 이용해서 kaggle API를 사용해도 됩니다.

Submission을 통한 성취감을 보여드리고 싶었는데 시시하게 끝나서 죄송합니다. 하지만, 이 모든 과정을 따라 오셨다면 이제 다른 대회나 프로젝트를 해도 전부 성공하실 수 있을 것입니다.